In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pickle

In [2]:
file_ratings_path = "data/movies/ratings.csv"

In [3]:
def missing_values_analysis(data):
    na_columns = [col for col in data.columns if data[col].isnull().sum() > 0]
    n_miss = data[na_columns].isnull().sum().sort_values(ascending=True)
    ratio = (data[na_columns].isnull().sum() / data.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df


In [4]:
def check_df(data, row_num=5, col_num=10):
    print("*************** Dataset Shape ***************")
    print("No. of Rows:", data.shape[0], "\nNo. of Columns:", data.shape[1])
    print("*************** Dataset Information ***************")
    print(data.info())
    print("*************** Types of Columns ***************")
    print(data.dtypes)
    print(f"*************** First {row_num} Rows ***************")
    print(data.iloc[:row_num,:col_num])
    print(f"*************** Last {row_num} Rows ***************")
    print(data.iloc[-row_num:,:col_num])
    print("*************** Summary Statistics of The Dataset ***************")
    print(data.describe([0.10, 0.25, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99]).T)
    print("*************** Dataset Missing Values Analysis ***************")
    print(missing_values_analysis(data))


In [5]:
rating = pd.read_csv(file_ratings_path)
df = rating

In [6]:
check_df(df)

*************** Dataset Shape ***************
No. of Rows: 3908657 
No. of Columns: 4
*************** Dataset Information ***************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3908657 entries, 0 to 3908656
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
 3   tstamp   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 119.3+ MB
None
*************** Types of Columns ***************
userId       int64
movieId      int64
rating     float64
tstamp      object
dtype: object
*************** First 5 Rows ***************
   userId  movieId  rating               tstamp
0     206     4803     4.0  2003-04-07 13:52:01
1    5073    72731     4.0  2020-02-19 16:07:53
2    4739    91653     4.0  2020-12-28 15:35:58
3     535     3005     3.0  2008-12-26 05:38:11
4     465     4776     3.0  2008-08-13 20:22:36
*************** Last 5 Rows ***************
         userId  movieId  rating 

In [52]:
def create_user_movie_df(dataframe):
    # comment
    comment_counts = pd.DataFrame( df['movieId'].value_counts())
    print(comment_counts)

    rare_movies = comment_counts[comment_counts["movieId"] <= 10].index
    print(rare_movies)

    common_movies = df[~df["movieId"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"],columns=["movieId"], values="rating")
    return user_movie_df

In [53]:
user_movie_df = create_user_movie_df(df)

        movieId
79132      5506
2571       5347
2959       5101
58559      4975
318        4945
...         ...
261523        1
254488        1
267804        1
108683        1
215387        1

[68044 rows x 1 columns]
Int64Index([ 77990,  71765, 147057, 223922,   8388, 185315,  79773, 135280,
            190155, 143506,
            ...
            122025, 246818, 166978, 125435, 192607, 261523, 254488, 267804,
            108683, 215387],
           dtype='int64', length=47854)


In [54]:
user_movie_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,267424,267458,267654,268732,268744,268806,269226,269820,269872,270350
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,3.0,NaN,4.5,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,3.0,3.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,3.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


In [55]:
def user_based_recommender(random_user, user_movie_df, ratio=60, cor_th=0.65, score=3.5):
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    # Chọn phim mà người dùng đã xem
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist() # select watched and rated movie by random_user
    print(f"Số lượng bộ phim mà người dung ID: [{random_user}] đã xem là: {len(movies_watched)}")

    # ===== Truy cập dữ liệu và id của những người dùng khác đang xem cùng một bộ phim ===== #
    movies_watched_df = user_movie_df[movies_watched]
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]
    perc = len(movies_watched) * ratio / 100
    # ===== Chọn những người dùng khác đã xem cùng một bộ phim ===== #
    users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"] #select other users who watched same movies
    # print(users_same_movies)


    # ===== Xác định người dùng có hành vi giống nhất với người dùng để được đề xuất ===== #
    # Để làm điều này, chúng tôi sẽ thực hiện 3 bước:
    #   Chúng tôi sẽ thu thập dữ liệu của Người dùng của chúng tôi và những người dùng khác.
    #   Chúng ta sẽ tạo mối tương quan df.
    #   Chúng tôi sẽ tìm những người dùng tương tự nhất (Top Users)

    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                          random_user_df[movies_watched]])
    # print(final_df.head())
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=["corr"])
    corr_df.index.names = ['user_id_1', 'user_id_2']
    corr_df = corr_df.reset_index()
    # print(corr_df.head())

    top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= cor_th)][
        ["user_id_2", "corr"]].reset_index(drop=True) # select similar users have correlation over cor_th on random_user
    top_users = top_users.sort_values(by='corr', ascending=False)
    top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
    rating = pd.read_csv(file_ratings_path)
    top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
    # ===== Tính điểm đề xuất trung bình có trọng số ===== #
    top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating'] # calculate rating*corr score
    # print(top_users_ratings.head())
    # print(top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"}).head())

    # ===== Thuật toán đề xuất film ===== #
    recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
    recommendation_df = recommendation_df.reset_index()
    # print(recommendation_df)
    # print(recommendation_df[recommendation_df["weighted_rating"] > 3.5])
    movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > score].sort_values("weighted_rating", ascending=False)

    return movies_to_be_recommend[["movieId"]]

In [56]:
def ShowAnswer(UserID = 0, cor_th = 0.70, score = 4):
  print("=============== Thông tin ===============")
  print("                                         ")
  print(f"Users ID: \t {UserID}")
  print(f"Score : \t {score}")
  print(f"cor_th: \t {cor_th}")
  print("                                         ")
  print("=========== Phim được đề xuất ===========")
  print("                                         ")
  print(user_based_recommender(UserID, user_movie_df, cor_th=cor_th, score=score))


In [57]:
ShowAnswer(UserID=1231, cor_th= 0.9, score=4)

=============== Thông tin ===============
                                         
Users ID: 	 1231
Score : 	 4
cor_th: 	 0.9
                                         
=========== Phim được đề xuất ===========
                                         
Số lượng bộ phim mà người dung ID: [1231] đã xem là: 276
     movieId
88      2353
96      2530
212    70451
107     2916
222    84152
..       ...
116     3519
113     3372
104     2717
103     2716
279   208737

[151 rows x 1 columns]
